The data size is huge, and thus it takes very long to get a submission score if we do training and inference in a single notebook. One way to avoid this long-waiting time is to split training and inference in separate notebooks. Here I demonstrate how to do it.

In this notebook only training is done.

This notebook is heavily based on 

- https://www.kaggle.com/lgreig/simple-lgbm-baseline
- https://www.kaggle.com/jsylas/riiid-lgbm-starter

Please upvote these notebooks too.

# Preprocess

In [ ]:
# Used most of coding from this kernel 
import optuna
import lightgbm as lgb
import pickle

import riiideducation
import dask.dataframe as dd
import  pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

# Config

In [ ]:
START_IDX = 80000000

# Load data

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                usecols=[1, 2, 3,4,7,8,9], dtype={'timestamp': 'int64', 'user_id': 'int32' ,'content_id': 'int16','content_type_id': 'int8','answered_correctly':'int8','prior_question_elapsed_time': 'float32','prior_question_had_explanation': 'boolean'}
              )
train = train[train.content_type_id == False]
#arrange by timestamp


train = train.sort_values(['timestamp'], ascending=True)

train.drop(['timestamp','content_type_id'], axis=1,   inplace=True)

results_c = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

results_u = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']

In [ ]:
#reading in question df
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',
                            usecols=[0,1, 3,4],
                            dtype={'question_id': 'int16',
                              'part': 'int8','bundle_id': 'int8','tags': 'str'}
                          )
tag = questions_df["tags"].str.split(" ", n = 10, expand = True) 
tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']

questions_df =  pd.concat([questions_df,tag],axis=1)
questions_df['tags1'] = pd.to_numeric(questions_df['tags1'], errors='coerce')
questions_df['tags2'] = pd.to_numeric(questions_df['tags2'], errors='coerce')
questions_df['tags3'] = pd.to_numeric(questions_df['tags3'], errors='coerce')
questions_df['tags4'] = pd.to_numeric(questions_df['tags4'], errors='coerce')
questions_df['tags5'] = pd.to_numeric(questions_df['tags5'], errors='coerce')
questions_df['tags6'] = pd.to_numeric(questions_df['tags6'], errors='coerce')

In [ ]:
X = train.iloc[START_IDX:,:]
X['prior_question_had_explanation'].fillna(False, inplace=True)
X = pd.merge(X, results_u, on=['user_id'], how="left")
X = pd.merge(X, results_c, on=['content_id'], how="left")
X = pd.merge(X, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')

X=X[X.answered_correctly!= -1 ]
X=X.sort_values(['user_id'])
Y = X[["answered_correctly"]]
X = X.drop(["answered_correctly"], axis=1)

In [ ]:
print('Training data shrinked from {:,} records ({:,} users) to {:,} records ({:,} users).'.format(train.shape[0], train['user_id'].nunique(), X.shape[0], X['user_id'].nunique()))

# Model fitting
I use a set of tuned hyperparameters used in my another kernel: [LGB hyperparameter tuning](https://www.kaggle.com/code1110/riiid-lgb-hyperparameter-tuning).

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])
X.head()

X = X[['answered_correctly_user', 'answered_correctly_content', 'sum','bundle_id','part','prior_question_elapsed_time','prior_question_had_explanation_enc','tags1','tags2','tags3']] 
X.fillna(0.5,  inplace=True)

from  sklearn.model_selection import train_test_split
Xt, Xv, Yt, Yv = train_test_split(X, Y, test_size = 0.2, shuffle=False)

In [ ]:
lgb_train = lgb.Dataset(Xt, Yt)
lgb_eval = lgb.Dataset(Xv, Yv)

params = {
    'num_leaves': 498,
    'max_depth': 16,
    'min_child_weight': 12,
    'feature_fraction': 0.9475614419516545,
    'bagging_fraction': 0.8023416987117281,
    'bagging_freq': 1,
    'min_child_samples': 79,
    'lambda_l1': 0.001840839009120818,
    'lambda_l2': 2.209995089627088e-06,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.04,
    'early_stopping_rounds': 10,
    'is_unbalance': True
}

In [ ]:
model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=1000)

# Feature importance

In [ ]:
# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('fivethirtyeight')

fi = pd.DataFrame()
fi['features'] = X.columns.values.tolist()
fi['importance'] = model.feature_importance(importance_type="gain")

sns.barplot(x='importance', y='features', data=fi.sort_values(by='importance', ascending=True))

# Validation Score

In [ ]:
y_pred = model.predict(Xv)
y_true = np.array(Yv)

print('CV score (AUC) = {}'.format(roc_auc_score(y_true, y_pred)))

# Save Model
We can simply use pickle:)

In [ ]:
file = 'trained_model.pkl'
pickle.dump(model, open(file, 'wb'))
print('Trained LGB model was saved!')

# Predictions

In [ ]:
# env = riiideducation.make_env()

In [ ]:
# iter_test = env.iter_test()
# for (test_df, sample_prediction_df) in iter_test:
#     test_df = test_df.sort_values(['user_id','timestamp'], ascending=False)
#     test_df['answer_time'] = test_df.groupby(['user_id'])['prior_question_elapsed_time'].shift(1)
    
#     test_df = pd.merge(test_df, results_u, on=['user_id'],  how="left")
#     test_df = pd.merge(test_df, results_c, on=['content_id'],  how="left")    
#     test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')    
#     test_df['answered_correctly_user'].fillna(0.5, inplace=True)
#     test_df['answered_correctly_content'].fillna(0.5, inplace=True)
#     test_df['sum'].fillna(0, inplace=True)
#     test_df['prior_question_had_explanation'].fillna(False, inplace=True)
#     test_df["prior_question_had_explanation_enc"] = lb_make.fit_transform(test_df["prior_question_had_explanation"])
#     test_df['answered_correctly'] =  model.predict(test_df[['answered_correctly_user', 'answered_correctly_content', 'sum','bundle_id','part','prior_question_elapsed_time','prior_question_had_explanation_enc',
#                                                            'tags1','tags2','tags3']])
#     env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])